In [145]:
from unittest.mock import inplace

import pandas as pd
import datetime
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [146]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

In [147]:
df.head()

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
Date,,,,,,,,,,,,,,,,,,,,,
1999-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-01,1.13,1.00,1.00,1.00,1.03,1.10,1.01,1.03,1.00,1.03,...,1.00,1.00,1.00,0.93,1.00,0.99,1.00,1.00,1.00,0.84
1999-12-01,1.11,1.00,1.00,1.00,0.92,1.02,1.10,1.00,1.00,0.97,...,1.00,1.00,1.00,1.18,1.00,1.01,1.00,1.00,1.00,0.93
2000-01-01,1.02,1.00,1.00,1.00,0.96,0.88,0.89,0.73,1.00,1.00,...,1.00,1.00,1.00,0.85,1.00,1.22,1.00,1.00,1.00,0.93
2000-02-01,1.00,1.01,0.81,1.23,1.08,0.96,1.25,1.95,1.18,0.93,...,1.03,0.94,1.07,1.04,1.47,1.97,1.13,0.95,1.01,0.94


### Ticker to index

In [148]:
df_T = df.copy(True)
df_T.rename_axis('ticker', inplace=True)
df_T = df_T.T
df_T.head(5)

ticker,1999-10-01,1999-11-01,1999-12-01,2000-01-01,2000-02-01,2000-03-01,2000-04-01,2000-05-01,2000-06-01,2000-07-01,...,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01
RS1.L,NaN,1.13,1.11,1.02,1.00,1.00,1.00,1.00,1.00,1.01,...,1.11,0.96,0.96,0.96,1.01,0.95,1.00,1.19,0.96,0.99
KE,NaN,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,...,1.09,0.88,0.95,0.96,0.97,1.10,0.96,1.08,0.78,0.95
TEG.DE,NaN,1.00,1.00,1.00,0.81,1.12,1.00,0.90,1.12,1.00,...,1.05,0.99,0.86,1.12,1.06,1.06,0.96,1.02,1.06,1.10
LEG.DE,NaN,1.00,1.00,1.00,1.23,1.03,1.00,1.00,1.00,1.00,...,1.13,0.98,0.88,1.17,1.00,1.02,0.97,1.06,1.08,1.08
SCS,NaN,1.03,0.92,0.96,1.08,0.98,0.99,1.04,1.39,1.01,...,1.10,0.94,1.09,0.95,0.92,1.14,0.95,1.12,0.98,0.93


### Dates as row

In [149]:
df.loc['1999-11-01', 'RS1.L']

1.13

In [150]:
dt = datetime.datetime.strptime('2005-11-01', "%Y-%m-%d")
df.loc[(dt - pd.DateOffset(months=12)).strftime('%Y-%m-%d'), 'SCS']

1.02

In [151]:
# Define Function:
def getvalue(data, date, ticker, months):
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    try:
        m = months * -1 if months > 0 else months
        offset = pd.DateOffset(months=m) if months > 0 else - pd.DateOffset(months=months)
        return df.loc[(dt + offset).strftime('%Y-%m-%d'), ticker]
    except KeyError:
        return None

In [152]:
df_tabular = pd.DataFrame(columns=['ticker', 'month', 'year', 'monthly_return'])

tickers = df.columns
dates = df.index

ticker_column = []
month_column = []
year_column = []
monthly_return_column_minus12 = []
monthly_return_column_minus11 = []
monthly_return_column_minus10 = []
monthly_return_column_minus9 = []
monthly_return_column_minus8 = []
monthly_return_column_minus7 = []
monthly_return_column_minus6 = []
monthly_return_column_minus5 = []
monthly_return_column_minus4 = []
monthly_return_column_minus3 = []
monthly_return_column_minus2 = []
monthly_return_column_minus1 = []
monthly_return_column = []
monthly_return_column_plus1 = []

for i, ticker in enumerate(tickers):
    for j, date in enumerate(dates):
        ticker_column.append(ticker)
        #
        dt = datetime.datetime.strptime(date, "%Y-%m-%d")
        month_column.append(dt.month)
        year_column.append(dt.year)
        #
        monthly_return = df.iloc
        monthly_return_column_minus12.append( getvalue(df, date, ticker, -12) )
        monthly_return_column_minus11.append( getvalue(df, date, ticker, -11) )
        monthly_return_column_minus10.append( getvalue(df, date, ticker, -10) )
        monthly_return_column_minus9.append( getvalue(df, date, ticker, -9) )
        monthly_return_column_minus8.append( getvalue(df, date, ticker, -8) )
        monthly_return_column_minus7.append( getvalue(df, date, ticker, -7) )
        monthly_return_column_minus6.append( getvalue(df, date, ticker, -6) )
        monthly_return_column_minus5.append( getvalue(df, date, ticker, -5) )
        monthly_return_column_minus4.append( getvalue(df, date, ticker, -4) )
        monthly_return_column_minus3.append( getvalue(df, date, ticker, -3) )
        monthly_return_column_minus2.append( getvalue(df, date, ticker, -2) )
        monthly_return_column_minus1.append( getvalue(df, date, ticker, -1) )
        monthly_return_column.append(df.loc[date, ticker])
        monthly_return_column_plus1.append( getvalue(df, date, ticker, 1) )


df_tabular = pd.DataFrame(data={
    'ticker': ticker_column,
    'month': month_column,
    'year': year_column,
    'm_return(t-12)': monthly_return_column_minus12,
    'm_return(t-11)': monthly_return_column_minus11,
    'm_return(t-10)': monthly_return_column_minus10,
    'm_return(t-9)': monthly_return_column_minus9,
    'm_return(t-8)': monthly_return_column_minus8,
    'm_return(t-7)': monthly_return_column_minus7,
    'm_return(t-6)': monthly_return_column_minus6,
    'm_return(t-5)': monthly_return_column_minus5,
    'm_return(t-4)': monthly_return_column_minus4,
    'm_return(t-3)': monthly_return_column_minus3,
    'm_return(t-2)': monthly_return_column_minus2,
    'm_return(t-1)': monthly_return_column_minus1,
    'm_return(t)': monthly_return_column,
    'm_return_target(t+1)': monthly_return_column_plus1
})

In [153]:
df_tabular

,ticker,month,year,m_return(t-12),m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),m_return(t-4),m_return(t-3),m_return(t-2),m_return(t-1),m_return(t),m_return_target(t+1)
0,RS1.L,10,1999,1.00,1.00,1.0,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11,1.13,NaN,NaN
1,RS1.L,11,1999,1.00,1.00,1.0,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11,1.13,NaN
2,RS1.L,12,1999,1.00,1.00,1.0,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11,1.13
3,RS1.L,1,2000,1.01,1.00,1.0,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11
4,RS1.L,2,2000,1.00,1.01,1.0,1.00,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495895,UNP,5,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98,0.98,0.96
495896,UNP,6,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98,0.98
495897,UNP,7,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98
495898,UNP,8,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09


In [154]:
df_tabular.loc[df_tabular['year'] >= 2021]

,ticker,month,year,m_return(t-12),m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),m_return(t-4),m_return(t-3),m_return(t-2),m_return(t-1),m_return(t),m_return_target(t+1)
255,RS1.L,1,2021,0.92,1.00,1.07,1.04,1.01,1.05,1.00,1.04,0.93,1.07,1.02,1.11,1.01,1.09
256,RS1.L,2,2021,0.89,0.92,1.00,1.07,1.04,1.01,1.05,1.00,1.04,0.93,1.07,1.02,1.11,1.01
257,RS1.L,3,2021,1.10,0.89,0.92,1.00,1.07,1.04,1.01,1.05,1.00,1.04,0.93,1.07,1.02,1.11
258,RS1.L,4,2021,0.97,1.10,0.89,0.92,1.00,1.07,1.04,1.01,1.05,1.00,1.04,0.93,1.07,1.02
259,RS1.L,5,2021,0.92,0.97,1.10,0.89,0.92,1.00,1.07,1.04,1.01,1.05,1.00,1.04,0.93,1.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495895,UNP,5,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98,0.98,0.96
495896,UNP,6,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98,0.98
495897,UNP,7,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98
495898,UNP,8,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09


### Ticker Labeling

In [155]:
le = LabelEncoder()
df_tabular['ticker_label'] = le.fit_transform(df_tabular['ticker'])
df_tabular.drop(columns=['ticker'], inplace=True)

### Train & Test

In [156]:
train, test = df_tabular.loc[df_tabular['year'] < 2021], df_tabular.loc[df_tabular['year'] >= 2021]


#### Remove NaN

In [165]:
train = train.dropna()

#### Input & Target

In [166]:
target_key = 'm_return_target(t+1)'

columns = train.columns.to_list()
columns.remove(target_key)
#
X_train = train[columns]
y_train = train[[target_key]]

X_test = test[columns]
y_test = test[[target_key]]

### Save Dataframe

In [168]:
X_train.to_csv('../../../data/train_test/df_tabular_xtrain.csv')
y_train.to_csv('../../../data/train_test/df_tabular_ytrain.csv')
X_test.to_csv('../../../data/train_test/df_tabular_xtest.csv')
y_test.to_csv('../../../data/train_test/df_tabular_ytest.csv')